<a href="https://colab.research.google.com/github/rayyirahmaid/Optic-Cup-Segmentation-with-K-Means-Clustering/blob/main/K_Means_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the image
image_path = '/content/drive/MyDrive/Tugas_Besar/drishtiGS_098.png'
A = cv2.imread(image_path)

In [ ]:
if A is None:
    print(f"There is no picture: {image_path}")
else:
    # SEGMENTATION
    # Channel Extraction (Green channel)
    gray = A[:,:,1]

    # K-means Clustering
    pixels = gray.reshape((-1, 1))
    pixels = np.float32(pixels) / 255.0

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    k = 7
    _, labels, (centers) = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers * 255)
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(gray.shape)

    # Compute mean intensity & find highest
    mean_intensity = [np.mean(segmented_image[labels.flatten() == i]) for i in range(k)]
    chosen_cluster_label = np.argmax(mean_intensity)

    # Extract chosen cluster
    chosen_cluster_pixels = (labels.reshape(gray.shape) == chosen_cluster_label)
    binary_mask = np.uint8(chosen_cluster_pixels) * 255

    # Morphological operation
    se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (50, 50))
    ED = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, se)

    # POST-PROCESSING
    BW = ED
    BW = cv2.morphologyEx(BW, cv2.MORPH_CLOSE, np.ones((3,3),np.uint8))
    BW = cv2.copyMakeBorder(BW, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
    BW = cv2.bitwise_not(BW)

    num_labels, labels_im = cv2.connectedComponents(BW)
    if num_labels > 1:
        component_sizes = np.bincount(labels_im.ravel())
        largest_component = labels_im == np.argmax(component_sizes[1:]) + 1
        BW = largest_component.astype(np.uint8)

    # Calculate centroid & fit ellipse
    contours, _ = cv2.findContours(BW, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    plt.figure(figsize=(8, 8))
    plt.imshow(cv2.cvtColor(A, cv2.COLOR_BGR2RGB))
    plt.title('OC Segmentation with K-Means')

    if contours:
        cnt = max(contours, key=cv2.contourArea)
        (x, y), (MA, ma), angle = cv2.fitEllipse(cnt)

        # Calculate ellipse lines
        theta = np.linspace(0, 2 * np.pi, 100)
        x_ellipse = x + MA/2 * np.cos(theta) * np.cos(np.deg2rad(angle)) - ma/2 * np.sin(theta) * np.sin(np.deg2rad(angle))
        y_ellipse = y + MA/2 * np.cos(theta) * np.sin(np.deg2rad(angle)) + ma/2 * np.sin(theta) * np.cos(np.deg2rad(angle))

        plt.plot(x_ellipse, y_ellipse, 'b', linewidth=2)

    plt.axis('off')
    plt.show()